In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import fiona

# set working directory
os.chdir('/Users/nkssssss/Desktop/MINES/COGCC-Risk-Analysis/Data')

/var/folders/p3/gs65vvgs1wd90mnxb3q3hvy80000gp/T/ipykernel_23740/3426520483.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
FlowlineSpreadsheet_Mines = pd.read_excel('FlowlineSpreadsheet_Mines.xlsx')
Flowline_Related_Spills = pd.read_excel('Flowline-Related Spills (2022).xlsx')

In [6]:
print(FlowlineSpreadsheet_Mines.info())
print(FlowlineSpreadsheet_Mines.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21942 entries, 0 to 21941
Data columns (total 88 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   DOC_NUM               21942 non-null  int64  
 1   FORM_NUM              21942 non-null  int64  
 2   RECEIVE_DATE          21942 non-null  object 
 3   OPERATOR_NUM          21942 non-null  int64  
 4   COMPANY_NAME          21942 non-null  object 
 5   SUBMIT_DATE           21942 non-null  object 
 6   STATUS_DATE           21942 non-null  object 
 7   FACILITY_ID           591 non-null    float64
 8   STATUS                21942 non-null  object 
 9   USER_NAME             21942 non-null  object 
 10  DATA_ENTRY_DATE       21942 non-null  object 
 11  DATA_ENTRY_OP         21867 non-null  object 
 12  LOCATION_ID           21928 non-null  float64
 13  LAST_USER             21942 non-null  object 
 14  LAST_UPDATE_USER      21942 non-null  object 
 15  LAST_UPDATE_DATE   

In [7]:
print(Flowline_Related_Spills.info())
print(Flowline_Related_Spills.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1726 entries, 0 to 1725
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   trkg_num                  1726 non-null   int64  
 1   Operator Name             1726 non-null   object 
 2   operator_num              1726 non-null   object 
 3   incident_date             1726 non-null   object 
 4   facility_type             1675 non-null   object 
 5   Spill_Desc                1726 non-null   object 
 6   county                    1726 non-null   object 
 7   Spill Type                880 non-null    object 
 8   Root Cause                1005 non-null   object 
 9   Preventative Measure      971 non-null    object 
 10  Root Cause Type           1726 non-null   object 
 11  Detailed Root Cause Type  1581 non-null   object 
 12  Long                      1537 non-null   float64
 13  Lat                       1537 non-null   float64
 14  oil_spil

In [1]:
# %%
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# %%
# %%
a = pd.read_excel("./Flowline-Related Spills (2022).xlsx")
a

# %%
b = pd.read_excel("./FlowlineSpreadsheet_Mines.xlsx")
b

# %%
a_ = a.sample(frac=1.00)
b_ = b.sample(frac=1.00)
# %%
a_
# %%
b_
# %% [markdown]
# The DOC_NUM and trkg_num are disjoint so they must not be join columns
# %%
b_id = set(b_.DOC_NUM)
a_id = set(a_.trkg_num)
b_id.intersection(a_id)
# %%

# %%
rect = b_
points = a_
# %% [markdown]
# count how many of points are within the rectangles

# %%

rect


# %%

# [
#     in_rectangle()
# ]


# %%
r_lat = rect.LAT
p_lat = points.Lat.dropna()
r_lat.describe()
# %%
p_lat.describe()

p_lat.hist()
# %%
set(r_lat).intersection(set(p_lat))
# %% [markdown]
# the lat column is a red herring! there are no latitude values in common
# %%
np.nan == np.nan
# %%
rect.LONG.hist()
# %% [markdown]
# find which points _belong_ to each rectangle

# %%

# rect -> id
# point -> id2
# id -> id2

# (id, id2)

def in_rectangle(rect, point, e=0):
    assert len(rect) == 4
    assert len(point) == 2
    [lat, long] = point
    [lat_start, lat_end, long_start, long_end] = rect

    # return (lat_start <= lat <= lat_end ) and (long_start <= long <= long_end)
    # within ε of b means `b - ε <= b <= b + ε`

    # compute the distance from the center of the circle to each of the edges of the rectangle
    # if they are 

    # return min([
    #     ((( lat_start - lat )**2 + (long - long_start) **2)  **.5) ,
    #     ((( lat_start - lat )**2 + (long - long_end  ) **2)  **.5) ,
    #     ((( lat_end   - lat )**2 + (long - long_end  ) **2)  **.5) ,
    #     ((( lat_end   - lat )**2 + (long - long_start) **2)  **.5) 
    # ])  <= e

    return ( lat_start -e <= lat <= lat_end + e) and (long_start -e <= long <= long_end + e)

# check if point is within rectangle

# def in_rectangle(rect, point):
#     assert len(rect) == 4
#     assert len(point) == 2
#     [x, y] = point
#     [x_0, x_1, y_0, y_1] = rect

#     return (x_0 <= x <= x_1 ) and (y_0 <= y <= y_1)


tests = [
    in_rectangle([-1, 1, -1, 1], [0,0]),
    not in_rectangle([-1, 1, -1, 1], [-1.01,0], e=0.0),
    in_rectangle([-1, 1, -1, 1], [-1.01,0], e=0.1),
    not in_rectangle([-1, 1, -1, 1], [-2,0]),
    not in_rectangle([-1, 1, -1, 1], [-2,0], e=0.002),
]
print(tests)
assert all(tests)

# put ID2 in the df containing id1
# df.map
rect

# %%
l_s = list(rect.STARTLAT.values)
l_e = list(rect.ENDLAT.values)
ln_s = list(rect.STARTLONG.values)
ln_e = list(rect.ENDLONG.values)
rect_idx = list(rect.index)


l = list(points.Lat.values)
ln = list(points.Long.values)
pt_idx = list(points.index)

match_ids = dict()

# TODO : change to find closest rectangle
for r_id, r in zip(rect_idx, zip(l_s, l_e, ln_s, ln_e)):
    for p_id, p in zip(pt_idx, zip(l, ln)):

        if any([np.isnan(e) for e in r]):
            continue

        if in_rectangle(rect=r, point=p):
            match_ids[r_id] = p_id

print(match_ids)
# %%


# %%
def test():
    (r, pt) = 2551, match_ids[2551]
    rect_record = rect.loc[r] 
    # rect_record.START_LAT
    r_list = [
        rect_record.STARTLAT,
        rect_record.ENDLAT,
        rect_record.STARTLONG,
        rect_record.ENDLONG
    ]

    pt_record = points.loc[pt]
    
    pt_list = [
        pt_record.Lat,
        pt_record.Long
    ]
    print(r_list, pt_list)
    return in_rectangle(rect=r_list, point=pt_list)

test()
# %%

# rectangle to point
rect_id_to_point_id = match_ids

join_col = rect.index.map(lambda x: rect_id_to_point_id.get(x) or np.nan)

pd.Series(join_col).describe()

rect['join_col'] = join_col 
points['join_col'] = points.index
# %%
merged = pd.merge(rect, points, how='inner', on="join_col", copy=True)

merged.to_excel("a.xlsx")
# %%
l
# %%
ln
# %%
np.isnan(np.nan)
# %%.

ModuleNotFoundError: No module named 'pandas'